In [1]:
import sys
sys.path.append("../")
from src.utils.data_util import biocaser2text
from src.eval_method import CrossValidation
from src.experiment import Experiment
from src.metrics import Accuracy,Precision,Recall,F1Score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from src.preprocess import BasicTextPreprocess,BasicTextFeatureExtraction

In [2]:
## this should load a dataframe, which col1 being the documents and col2 being the labels.
data_file = "../data/biocaster/BioCaster.3.xml"
data_df = biocaser2text(data_file)
map_dic = {"negative":0,"positive":1}
data_df.labels = data_df.labels.apply(lambda x: map_dic[x])
data_df.head(3)

parse biocaser data from ../data/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0
1,\nTyphoid outbreak in Agusan del Sur town unde...,1
2,\n Typhoid Outbreak In Central Nepal November...,1


In [3]:
bp = BasicTextPreprocess()

bfe = BasicTextFeatureExtraction(["tfidf","bow","bigram",["tfidf","bigram"],["bow","bigram"],["tfidf","trigram"],["bow","trigram"],["tfidf","bigram","trigram"]])

cv = CrossValidation(n_splits=10, n_repeats=10, random_state=12345)

acc = Accuracy()
pre = Precision()
rec = Recall()
f1 = F1Score()

svm = LinearSVC(random_state=0, tol=1e-5,max_iter=5000)
svm.name = "svm"
dtree = DecisionTreeClassifier(random_state=0,criterion='entropy')
dtree.name="dtree"
bnb = BernoulliNB()
bnb.name = "BernoulliNB"
gnb = GaussianNB()
gnb.name = "GaussianNB"
mnb = MultinomialNB()
mnb.name = "MultinomialNB"

# chi2 feature selection
ch2 = SelectKBest(chi2, k=9000)
svm_ch2 = Pipeline([('sel', ch2), ('cls', svm)])
svm_ch2.name = "svm+chi2"

# chi2 feature selection
dtree_ch2 = Pipeline([('sel', ch2), ('cls', dtree)])
dtree_ch2.name = "dtree+chi2"

# chi2 feature selection
bnb_ch2 = Pipeline([('sel', ch2), ('cls', bnb)])
bnb_ch2.name = "BernoulliNB+chi2"

# chi2 feature selection
gnb_ch2 = Pipeline([('sel', ch2), ('cls', gnb)])
gnb_ch2.name = "GaussianNB+chi2"

# chi2 feature selection
mnb_ch2 = Pipeline([('sel', ch2), ('cls', mnb)])
mnb_ch2.name = "MultinomialNB+chi2"

In [4]:
exp = Experiment(
    data_df = data_df,
    preprocessor = bp,
    extractor = bfe,
    eval_method=cv,
    models=[svm,dtree,bnb,gnb,mnb,svm_ch2,dtree_ch2,bnb_ch2,gnb_ch2,mnb_ch2],
    metrics=[acc,pre,rec,f1],
    result_file = "../biocaser_result.csv",
    verbose = True
).run()

2it [00:00, 10.08it/s]

Found feature_columns: ['tfidf', 'bow', 'bigram', 'tfidf+bigram', 'bow+bigram', 'tfidf+trigram', 'bow+trigram', 'tfidf+bigram+trigram']
Experiment with svm model on tfidf feature.


100it [00:10,  9.50it/s]
1it [00:00,  6.00it/s]

total num of splits: 100
Create new result_file: ../biocaser_result.csv
saving result ...
   accuracy  precision    recall  f1_score  Train (s)  Test (s) model feature
0  0.917743   0.880836  0.902338  0.890168   0.097547  0.005657   svm   tfidf
Experiment with svm model on bow feature.


100it [00:16,  6.11it/s]
1it [00:00,  5.62it/s]

...

total num of splits: 100
../biocaser_result.csv  already exists, appending result to it
saving result ...
   accuracy  precision    recall  f1_score  Train (s)  Test (s)  \
0  0.860893   0.777802  0.879179  0.823384    0.27236  0.012205   

                model        feature  
0  MultinomialNB+chi2  tfidf+trigram  
Experiment with MultinomialNB+chi2 model on bow+trigram feature.


100it [00:42,  2.34it/s]
0it [00:00, ?it/s]

total num of splits: 100
../biocaser_result.csv  already exists, appending result to it
saving result ...
   accuracy  precision    recall  f1_score  Train (s)  Test (s)  \
0   0.90212   0.824694  0.939053  0.876759   0.415399  0.010624   

                model      feature  
0  MultinomialNB+chi2  bow+trigram  
Experiment with MultinomialNB+chi2 model on tfidf+bigram+trigram feature.


100it [00:45,  2.19it/s]

total num of splits: 100
../biocaser_result.csv  already exists, appending result to it
saving result ...
   accuracy  precision    recall  f1_score  Train (s)  Test (s)  \
0  0.835984    0.70418  0.968495  0.813918    0.43963  0.015777   

                model               feature  
0  MultinomialNB+chi2  tfidf+bigram+trigram  


In [8]:
data_df.iloc[1]["docs"]

'number told magandang libertad far declar bunawan confirm del municip abl sinc well fatal clarifi hospit dr doctor water ruth sur deep main town checkup said particularli fever pilipina place 30 servic agusan manag offic health improso outbreak document umaga 500 case control peopl gone sought wa close advis use went admit typhoid barangay resid sourc villag monday high deni ha'